In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy as sp
import csv
from torch.utils.data import DataLoader, TensorDataset
from qiskit import Aer, execute, QuantumCircuit
from qiskit.quantum_info import DensityMatrix, random_statevector
from IPython.display import clear_output

In [ ]:
# Functions
def create_circuit():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    return qc

# Function to measure in different bases
def measure_basis(qc, basis):
    if basis == 'ZZ':
        qc.measure_all()
    elif basis == 'XX':
        qc.h([0, 1])
        qc.measure_all()
    elif basis == 'YY':
        qc.sdg([0, 1])
        qc.h([0, 1])
        qc.measure_all()
    return qc

# Function to simulate measurements
def simulate_measurements(qc):
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, shots=1024).result()
    counts = result.get_counts()
    return counts

# Function to collect measurement data
def collect_data(statevector):
    bases = ['ZZ', 'XX', 'YY']
    data = []
    for basis in bases:
        qc = QuantumCircuit(2)
        qc.initialize(statevector, [0, 1])
        qc = create_circuit().compose(qc)
        qc = measure_basis(qc, basis)
        counts = simulate_measurements(qc)
        data.append(counts)
    return data

# Convert measurement counts to probabilities
def counts_to_probabilities(counts, num_qubits):
    shots = sum(counts.values())
    probabilities = {k: v / shots for k, v in counts.items()}
    # Ensure all measurement outcomes are present
    for i in range(2 ** num_qubits):
        key = format(i, f'0{num_qubits}b')
        if key not in probabilities:
            probabilities[key] = 0.0
    return probabilities

# Preprocess the training data
def preprocess_data(training_data, num_qubits):
    processed_data = []
    for sample in training_data:
        sample_data = []
        for counts in sample:
            probabilities = counts_to_probabilities(counts, num_qubits)
            sample_data.extend([probabilities[format(i, f'0{num_qubits}b')] for i in range(2 ** num_qubits)])
        processed_data.append(sample_data)
    return np.array(processed_data)

In [ ]:
# Generate training data
num_samples = 50000
num_qubits = 2
training_data = []
training_labels = []

for i in range(num_samples):
    statevector = random_statevector(2**num_qubits).data  # Generate random state vector
    data = collect_data(statevector)  # Collect measurement data for the current state
    qc = QuantumCircuit(2)
    qc.initialize(statevector, [0, 1])
    qc = create_circuit().compose(qc)  # Apply fixed circuit to the state
    state = execute(qc, Aer.get_backend('statevector_simulator')).result().get_statevector()
    density_matrix = DensityMatrix(state)  # Convert state vector to density matrix
    # Separate real and imaginary parts
    real_part = np.real(density_matrix.data).flatten()
    imag_part = np.imag(density_matrix.data).flatten()
    combined_data = np.concatenate((real_part, imag_part))
    training_data.append(data)
    training_labels.append(combined_data)

    clear_output(wait=True)
    print(f"Finished with sample number: {i}")

# Preprocess the training data
processed_training_data = preprocess_data(training_data, num_qubits)
training_labels = np.array(training_labels)

# Convert to PyTorch tensors
X_train = torch.tensor(processed_training_data, dtype=torch.float32)
y_train = torch.tensor(training_labels, dtype=torch.float32)

In [ ]:
np.savetxt(f'processed_training_data_{num_samples}.csv', processed_training_data, delimiter=',')
np.savetxt(f'training_labels_{num_samples}.csv', training_labels, delimiter=',')